# Multi-Label Classification Strategies
In this task you deal with multiclass classification problem for [Glass Classification Data](https://www.kaggle.com/uciml/glass). Lets load the dataset.

In [1]:
# если вы работаете в Colab то запустите эти строчки

#! wget https://raw.githubusercontent.com/aminovT/MADMO/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5%20%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5/data/glass.csv
  
# ! mkdir data

# ! mv glass.csv data

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/glass.csv')
X, y = data.drop('Type', axis=1), data.Type
data.sample(3)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
119,1.51652,13.56,3.57,1.47,72.45,0.64,7.96,0.0,0.00,2
98,1.51689,12.67,2.88,1.71,73.21,0.73,8.54,0.0,0.00,2
125,1.51872,12.93,3.66,1.56,72.51,0.58,8.55,0.0,0.12,2


In [32]:
X.shape

(214, 9)

In [3]:
y.value_counts()

2    76
1    70
7    29
3    17
5    13
6     9
Name: Type, dtype: int64

In [38]:
y.value_counts() / len(y)

2    0.355140
1    0.327103
7    0.135514
3    0.079439
5    0.060748
6    0.042056
Name: Type, dtype: float64

Признаки каждого стеклянного объекта соответствуют доле конкретного химического элемента в объекте. Целевая переменная соответствует типу стекла (6 классов).

В этой задаче вам необходимо эмпирически сравнить временную сложность и производительность нескольких стратегий мультиклассовой классификации для разных алгоритмов. Рассмотрим следующие алгоритмы:
* KNearestNeighbors (5 neighbors)
* Logistic Regression
* SVC \[Support Vector Classification\] (linear kernel)

Обратите внимание, что все эти алгоритмы по умолчанию поддерживают **multiclass labeling**. Тем не менее, сравните этот подход с **OneVSRest** и **OneVSOne** подходы, применяемые к этим алгоритмам. Точнее, для каждой пары (алгоритм, подход) выполните 5-кратную перекрестную проверку данных и выведите оценку проверки и время вычисления (в виде таблицы).


Обратите внимание, что набор данных является одновременно многоклассовым и несбалансированным, поэтому важно выбрать правильную оценку качества. Попробуйте разные показатели для оптимизации во время CV (например, точность, сбалансированная точность, f1, roc-auc).


После этого ответьте на следующие вопросы:
* Какой показатель вы бы выбрали для оптимизации во время перекрестной проверки и почему?
* Для каких алгоритмов использование подхода OneVSRest / OneVSOne обеспечивает значительно лучшую производительность без значительного увеличения времени вычислений?

In [5]:
# proper way to measure performance in modern Python! No time.time()!
# see https://docs.python.org/3/library/time.html#time.perf_counter
from time import perf_counter

# funct to properly display numpy arrays inline, use instead of print
from IPython.display import display

from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

In [8]:
from tqdm import tqdm_notebook

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [7]:
from sklearn.model_selection import cross_validate

In [21]:
import numpy as np

In [13]:
import psutil

In [15]:
psutil.cpu_count(), psutil.cpu_percent()

(4, 38.6)

In [73]:
# your code here
clf_list = [KNeighborsClassifier(5), LogisticRegression(), SVC(kernel='linear', probability=True)]
scoring = ['balanced_accuracy','f1_macro', 'f1_weighted', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted']
score = []

for clf in tqdm_notebook(clf_list):
    ovo_start = perf_counter()
    cv_ovo = cross_validate(OneVsOneClassifier(clf), X, y, cv=5, scoring=scoring[:-2], 
                                     verbose=True, n_jobs=2)
    ovo_finish = perf_counter()
    score.append({'classifier': clf, 
                  'mode': 'ovo',
                  'time': ovo_finish - ovo_start,  
#                   'roc auc': np.mean(cv_ovo['test_roc_auc_ovr']), 
                  'f1 macro': np.mean(cv_ovo['test_f1_macro']), 
                  'f1 weighted': np.mean(cv_ovo['test_f1_weighted']),  
                  'balanced accuracy': np.mean(cv_ovo['test_balanced_accuracy'])})

    ovr_start = perf_counter()
    cv_ovr = cross_validate(OneVsRestClassifier(clf), X, y, cv=5, scoring=scoring, 
                            verbose=True, n_jobs=2)
    ovr_finish = perf_counter()
    score.append({'classifier': clf, 
                  'mode':'ovr',
                  'time': ovr_finish - ovr_start,  
                  'roc auc ovr': np.mean(cv_ovr['test_roc_auc_ovr_weighted']),
                  'roc auc ovo': np.mean(cv_ovr['test_roc_auc_ovo_weighted']),
                  'f1 macro': np.mean(cv_ovr['test_f1_macro']), 
                  'f1 weighted': np.mean(cv_ovr['test_f1_weighted']),  
                  'balanced accuracy': np.mean(cv_ovr['test_balanced_accuracy'])})
    
    multi_start = perf_counter()
    cv_multi = cross_validate(clf, X, y, cv=5, scoring=scoring, 
                                         verbose=True, n_jobs=2)
    multi_finish = perf_counter()
    score.append({'classifier': clf, 
                  'mode': 'multi',
                  'time': multi_finish - multi_start,  
                  'roc auc ovr': np.mean(cv_multi['test_roc_auc_ovr_weighted']),
                  'roc auc ovo': np.mean(cv_multi['test_roc_auc_ovo_weighted']),
                  'f1 macro': np.mean(cv_multi['test_f1_macro']),  
                  'f1 weighted': np.mean(cv_multi['test_f1_weighted']),  
                  'balanced accuracy': np.mean(cv_multi['test_balanced_accuracy'])})

#     score.append({'Classifier': clf, 
#                   'OvO time': ovo_finish - ovo_start, 
#                   'OvR time': ovr_finish - ovr_start, 
#                   'Multiclass time': multi_finish - multi_start, 
#                   'OvO f1': np.mean(cv_ovo['test_f1_macro']), 
#                   'OvR f1': np.mean(cv_ovr['test_f1_macro']), 
#                   'Multiclass f1':np.mean(cv_multi['test_f1_macro']), 
#                   'OvO acc': np.mean(cv_ovo['test_balanced_accuracy']), 
#                   'OvR acc': np.mean(cv_ovr['test_balanced_accuracy']), 
#                   'Multiclass acc': np.mean(cv_multi['test_balanced_accuracy'])})
#                   ,
#                   'OvO ROC AUC OvO': cv_ovo['roc_auc_ovo_weighted'], 
#                   'OvR ROC AUC OvO': cv_ovr['roc_auc_ovo_weighted'], 
#                   'Multiclass ROC AUC OvO':cv_multi['roc_auc_ovo_weighted'], 
#                   'OvO ROC AUC OvR': cv_ovo['roc_auc_ovr_weighted'], 
#                   'OvR ROC AUC OvR': cv_ovr['roc_auc_ovr_weighted'], 
#                   'Multiclass ROC AUC OvR':cv_multi['roc_auc_ovr_weighted']})

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:   

[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.1s finished


In [74]:
score_df = pd.DataFrame(score)

In [75]:
score_df

,classifier,mode,time,f1 macro,f1 weighted,balanced accuracy,roc auc ovr,roc auc ovo
0,KNeighborsClassifier(),ovo,0.489774,0.522427,0.591219,0.554921,NaN,NaN
1,KNeighborsClassifier(),ovr,0.266410,0.535505,0.595097,0.575159,0.828614,0.843803
2,KNeighborsClassifier(),multi,0.124337,0.516398,0.577174,0.551071,0.828614,0.843803
3,LogisticRegression(),ovo,1.511244,0.392281,0.507087,0.434405,NaN,NaN
4,LogisticRegression(),ovr,0.699024,0.333318,0.497738,0.385298,0.767028,0.806815
5,LogisticRegression(),multi,0.255950,0.408289,0.524475,0.445040,0.782388,0.820278
6,"SVC(kernel='linear', probability=True)",ovo,0.342666,0.457824,0.538089,0.500298,NaN,NaN
7,"SVC(kernel='linear', probability=True)",ovr,0.370829,0.351663,0.431185,0.418829,0.735344,0.758036
8,"SVC(kernel='linear', probability=True)",multi,0.155746,0.445727,0.528091,0.481548,0.798101,0.826817
